# TVLA Testing for Crypto Validation

Supported setups:

SCOPES:

* OPENADC
* CWNANO

PLATFORMS:

* CWLITEARM
* CWLITEXMEGA
* CWNANO

This tutorial will perform a basic TVLA test. Here we are only using an unprotected software implementation, so there is little hope of passing the test! But this can demonstrate how the TVLA test can be useful for validating your crypto implementation.

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
CRYPTO_TARGET = 'TINYAES128C'
num_traces = 50
CHECK_CORR = False

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../hardware/victims/firmware/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2

## Capturing Power Traces

### Setup

We'll use some helper scripts to make setup and programming easier. If you're using an XMEGA or STM (CWLITEARM) target, binaries with the correct should be setup for you:

In [ ]:
%run "Helper_Scripts/Setup_Generic.ipynb"

In [ ]:
fw_path = "../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex".format(PLATFORM)

In [ ]:
cw.program_target(scope, prog, fw_path)

In addition, before we capture our traces, we'll need to create a ChipWhipserer project, since that's what Analyzer expects for an input:

In [ ]:
project = cw.create_project("projects/jupyter_tvla_sw.cwp", overwrite=True)

### Capturing Traces

Below you can see the capture loop. The main body of the loop loads some new plaintext, arms the scope, sends the key and plaintext, then finally records and our new trace into our trace class. We'll also keep track of our keys manually for checking our answer later.

In [ ]:
#Capture Traces
import chipwhisperer as cw
from tqdm import tnrange
import numpy as np
import time

N = 50  # Number of traces

ktp = cw.ktp.TVLATTest()
ktp.init(N)

keys = []
for i in tnrange(N, desc='Capturing traces'):
    key, text = ktp.next()  # TVLA T-Test changes PT between two options
    keys.append(key)
    trace = cw.capture_trace(scope, target, text, key)
    if trace is None:
        continue
    project.traces.append(trace)

Now that we have our traces, we need to tell the project that the traces are loaded and add them to the project's trace manager.

In [ ]:
#Save project file
project.save()

We're now done with the ChipWhisperer hardware, so we should disconnect from the scope and target:

In [ ]:
# cleanup the connection to the target and scope
scope.dis()
target.dis()

## Analysis

Alright! The following chunk of code does the entire TVLA algorithm itself, based on the recorded data:

In [ ]:
import holoviews as hv
hv.extension('bokeh')
import numpy as np
import scipy
import scipy.stats

project = cw.open_project('projects/jupyter_tvla_sw.cwp')
fixedpy = [0xda, 0x39, 0xa3, 0xee, 0x5e, 0x6b, 0x4b, 0x0d, 0x32, 0x55, 0xbf, 0xef, 0x95, 0x60, 0x18, 0x90]

testouts = []
num_traces = len(project.traces)
num_points = len(project.waves[0])
print(num_traces, num_points)
curve = hv.Curve([])

def do_the_ttvla(project, ntraces=-1):
    global curve, line
    if ntraces == -1:
        ntraces = int(num_traces / 2)
        
    if ntraces * 2 > num_traces:
        raise ValueError("Invalid ntraces")
    for g in range(0, 2):
        group = [(list(project.textins[i]) == fixedpy) for i in range(g*ntraces, g*ntraces+ntraces)]
        trace = np.zeros((ntraces, num_points))

        for n in range(g*ntraces, g*ntraces+ntraces):
            trace[n - g*ntraces][:] = project.waves[n]
                
        testout = welch_ttest(group, trace)
        curve *= hv.Curve(testout)
        testouts.extend(testout)
        
    curve *= hv.Path([(0, -4.5), (num_points, -4.5)]).opts(color="black") * \
    hv.Path([(0, 4.5), (num_points, 4.5)]).opts(color="black")

def welch_ttest(group, traces):
    import warnings
    # Compute Welch's t-statistic at each point in time
    # Here, group[] must only contain booleans (True/False)
    traces_true = traces[np.where(np.array(group))]
    traces_false = traces[np.where(~np.array(group))]
    
    if len(traces_true) == 0:
        traces_true  = np.array([[np.nan for _ in range(len(traces[0]))]])
    if len(traces_false) == 0:
        traces_false = np.array([[np.nan for _ in range(len(traces[0]))]])
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        ttrace = scipy.stats.ttest_ind(traces_true, traces_false, axis=0, equal_var=False)[0]
        
    return np.nan_to_num(ttrace) 
    
do_the_ttvla(project)
(curve).opts(height=600, width=600)

With multiple excursions beyond the 4.5 limit, you can see that the implementation easily fails! The only thing we haven't done right here is figure out *where* the actual crypto stards and ends. This is best done using a T-Test or similar targetting the input and output data of the crypto operation. In this example we didn't even cover the complete portion of the algorithm, but this is easier to do with hardware crypto.

## Tests

In [ ]:
max_leakage = 0
for point in np.abs(testouts):
    if point > max_leakage:
        max_leakage = point
assert max_leakage > 4.5, "Device passed test! Max leakage = {}".format(max_leakage)